In [1]:
# Dependencies
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets 
import pandas as pd
import time
import requests
from census import Census
#pd.set_option('display.max_rows', 30000)
# Census API Key
from config import census_key
c = Census(census_key, year=2015)

## Retrieve Crime Data from FBI Crime Database - UCR

In [2]:
#Store file path for crime data in a variable
crime_data = "../Crime Source Data/Offenses_Known_to_Law_Enforcement_by_State_by_City_2019.xls"

#Read data into dataframe
crime_data_df=pd.read_excel(crime_data)

#Extract the state into a list so all rows in "state" column can have a state value
state = crime_data_df["State"]
states=[]
for s in state:
    if pd.isnull(s):
        states.append(current_state)
    else:
        current_state = s
        states.append(current_state)
        
#Add newly created list into the dataframe and remove the old column
crime_data_df["NEW_STATE"] = states
clean_crime_data_df=crime_data_df[["NEW_STATE", "City", "Population", "Violent Crime"]] 
clean_crime_data_df=clean_crime_data_df.rename(columns={"NEW_STATE":"State"})

#Remove any trailing numbers at the end of state and city names
clean_crime_data_df["State"] = clean_crime_data_df["State"].map(lambda x: x.rstrip("1234567890"))
clean_crime_data_df["City"] = clean_crime_data_df["City"].map(lambda x: x.rstrip("1234567890"))

#Calculate the rate of violent crime per capita and remove any city where the violent crime is 0
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]
clean_crime_data_df["Violent Crime Rate"] = (clean_crime_data_df["Violent Crime"]/clean_crime_data_df["Population"])*100

clean_crime_data_df=clean_crime_data_df.sort_values("State")
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]

In [15]:
clean_crime_data_df=clean_crime_data_df[["State", "City", "Violent Crime Rate"]]
clean_crime_data_df.to_csv("2019_FBI_ViolentCrime.csv", index=False)


## Sort Out Maryland and Virginia Crime Rate

In [3]:
#Sort out the highest crime rate for each state
#variable to hold highest violent crime for each state
violent_crime=[]

#variable to track highest violent crime for each state
violent_crime_track=0

#variable to hold starting state for the loop
md='MARYLAND'
va='VIRGINIA'
fl='FLORIDA'
tx='TEXAS'
ny='NEW YORK'
nc= 'NORTH CAROLINA'
ca= 'CALIFORNIA'
ga='GEORGIA'
tn='TENNESSEE'
#variable to append city for highest crime rate
city=[]

#variable to append city for highest crime rate
state=[]

#Loop through each row in dataframe 
for index,row in clean_crime_data_df.iterrows():
    if (row['State'] == md) or (row['State'] == va) or (row['State'] == fl) or (row['State'] == tx) or (row['State'] == ny) or (row['State'] ==nc) or (row['State'] == ca) or (row['State'] == ga) or (row['State'] == tn):
            violent_crime.append(row["Violent Crime Rate"])
            city.append(row["City"])
            state.append(row["State"])
               
#Place newly created lists into dictionary and convert to DataFrame
highest_new_dict = {'State':state,'City':city,'Violent Crime Rate':violent_crime}
highest_new_df = pd.DataFrame(highest_new_dict)
highest_new_df.to_csv("2019_NineStates_Crime.csv", index=False)
highest_new_df

TypeError: append() takes exactly one argument (2 given)